In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange
import rosbags
import json
import open3d as o3d
import pandas as pd
import os
import copy
from preprocess import listFiles
from register import loadData
from scipy.spatial.transform import Rotation as R

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [46]:
def quat2RotationMatrix(q):
    """
    Convert a quaternion into a rotation matrix.
    
    Args:
    q (numpy.ndarray): A 4-element array representing the quaternion (w, x, y, z)
    
    Returns:
    numpy.ndarray: A 3x3 rotation matrix
    """
    w, x, y, z = q

    # Compute the rotation matrix elements
    R = np.array([
        [1 - 2*y*y - 2*z*z, 2*x*y - 2*z*w, 2*x*z + 2*y*w],
        [2*x*y + 2*z*w, 1 - 2*x*x - 2*z*z, 2*y*z - 2*x*w],
        [2*x*z - 2*y*w, 2*y*z + 2*x*w, 1 - 2*x*x - 2*y*y]
    ])
    return R


def transformPointCloud(point_cloud, translation, quaternion):
    """
    Apply translation and rotation to a point cloud.
    
    Args:
        point_cloud (np.ndarray): An n x 3 array representing the point cloud.
        translation (np.ndarray): A 1 x 3 array representing the translation vector.
        quaternion (np.ndarray): A 1 x 4 array representing the quaternion for rotation.
    
    Returns:
        np.ndarray: The transformed point cloud.
    """
    # Validate inputs
    assert point_cloud.shape[1] == 3, "Point cloud must be an n x 3 array."
    assert translation.shape == (3,), "Translation must be a 1 x 3 array."
    assert quaternion.shape == (4,), "Quaternion must be a 1 x 4 array."

    R = quat2RotationMatrix(quaternion)
    temp = R@(np.transpose(point_cloud))
    temp2 = np.transpose(temp) + translation

    return temp2


def quickVizReg(source, target):
    pcd1 = o3d.geometry.PointCloud()
    pcd2 = o3d.geometry.PointCloud()
    pcd1.points = o3d.utility.Vector3dVector(source)
    pcd2.points = o3d.utility.Vector3dVector(target)
    pcd1.paint_uniform_color([1, 0, 0])
    pcd2.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([pcd1,pcd2])

def quickVisualize(points):
    """ Visualize pointcloud from numpy array"""
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    o3d.visualization.draw_geometries([pcd])
    return

In [69]:
path = r'datasets/pose/box_plant2_pose.json'
with open(path,'r') as file:
    pose = json.load(file)

path = r'datasets/json/box_plant2/box_plant2_frame0_100.json'
with open(path,'r') as file:
    points1 = json.load(file)

path = r'datasets/json/box_plant2/box_plant2_frame600_700.json'
with open(path,'r') as file:
    points2 = json.load(file)

pose_keys = list(pose.keys())
point1_keys = list(points1.keys())
point2_keys = list(points2.keys())

In [79]:
frame1_idx = 5
frame2_idx = 690
offset = -5
point1 = np.array(points1[point1_keys[frame1_idx%100]])
point2 = np.array(points2[point2_keys[frame2_idx%100]])


translation1 = np.array(pose[pose_keys[frame1_idx+offset]]['translation'])
translation2 = np.array(pose[pose_keys[frame2_idx+offset]]['translation'])
rotation1 = pose[pose_keys[frame1_idx+offset]]['rotation']
rotation2 = pose[pose_keys[frame2_idx+offset]]['rotation']

In [81]:
points1_reg = transformPointCloud(point1, np.array(translation1), np.array(rotation1))
points2_reg = transformPointCloud(point2, np.array(translation2), np.array(rotation2))

In [82]:
quickVizReg(points1_reg, points2_reg)

In [87]:
path = r'testing.json'
with open(path,'r') as file:
    temp = json.load(file)

In [94]:
k = list(temp.keys())
a = temp[k[5]]
b = temp[k[400]]

In [95]:
quickVizReg(a,b)